# Pyramid Stereo Matching Network
Input | Output
--- | ---
![alt text](https://github.com/sony/nnabla-examples/raw/master/stereo-depth/PSMnet/results/0006.png) | ![alt text](https://github.com/sony/nnabla-examples/raw/master/stereo-depth/PSMnet/results/depth_sceneflow.png)
![alt text](https://github.com/sony/nnabla-examples/raw/master/stereo-depth/PSMnet/results/000000_10.png) | ![alt text](https://github.com/sony/nnabla-examples/raw/master/stereo-depth/PSMnet/results/depth_kitti.png)

This example interactively demonstrates [Pyramid Stereo Matching Network](https://arxiv.org/abs/1803.08669), a model for depth estimation.

# Preparation
Let's start by installing nnabla and accessing [nnabla-examples repository](https://github.com/sony/nnabla-examples). If you're running on Colab, make sure that your Runtime setting is set as GPU, which can be set up from the top menu (Runtime → change runtime type), and make sure to click **Connect** on the top right-hand side of the screen before you start.

In [ ]:
# May show warnings for newly imported packages if run in Colab default python environment.
# Please click the `RESTART RUNTIME` to run the following script correctly.
# The error message of conflicts is acceptable.
!pip install nnabla-ext-cuda116
!git clone https://github.com/sony/nnabla-examples.git

In [ ]:
%cd nnabla-examples/stereo-depth/PSMnet

We need to download the pretrained weights.

In [ ]:
# get PSMNet pretrained weights trained on Scene Flow Datasets.
!wget https://nnabla.org/pretrained-models/nnabla-examples/stereo-depth/psmnet_trained_sceneflow.h5

# Get images for inference
To use PSMNet, you need stereo vision images, in other words, **you need 2 images**, one is the left view, and the other is the right view. In this demo, we use images from [Scene Flow dataset](k.uni-freiburg.de/resources/datasets/SceneFlowDatasets.en.html) on which PSMNet is trained. First you need to download the dataset, but since the entire dataset is too large, we use the Example pack for instance. After downloading dataset, decompress the dataset.

In [ ]:
!wget --no-check-certificate https://lmb.informatik.uni-freiburg.de/resources/datasets/SceneFlow/assets/Sampler.tar.gz
!tar -xvf Sampler.tar.gz > /dev/null 2>&1

# Choose a left image

Run the below cell to choose a left view image.

In [ ]:
import glob
import ipywidgets as widgets
choices = sorted(glob.glob('Sampler/**/*.png', recursive=True))
left_images = [_ for _ in choices if "/left/" in _]
right_images = [_ for _ in choices if "/right/" in _]
left_image_path = widgets.Dropdown(options=left_images, value=left_images[0])
left_image_path

Next is the right view image. Running the following cell automatically finds the right view image corresponding to the left one.

In [ ]:
left_image_path = left_image_path.get_interact_value()
right_image_path = left_image_path.replace("/left/", "/right/")
assert right_image_path in right_images

Let's take a look at the chosen image (Here the left image is displayed).

In [ ]:
from IPython.display import Image, display
display(Image(left_image_path, width=960/2., height=512/2.))

# Run PSMNet for depth estimation

Now that we have input stereo paired images, We're ready to run inference script! Just execute the following cell to make it run (It may take time).

In [ ]:
!python inference.py --dataset SceneFlow --loadmodel ./psmnet_trained_sceneflow.h5 --save-nnp False -l $left_image_path -r $right_image_path

Once the inference is done, you should get the result image named "stereo_depth.png". Let's see the result and compare it with the input image.

In [ ]:
display(Image('stereo_depth.png', width=960/2., height=512/2.))
display(Image(left_image_path, width=960/2., height=512/2.))